In [ ]:
# https://maxhalford.github.io/blog/bayesian-linear-regression/

There are two types of machine learning methods - online and traditional machine learning.
Online model is dynamic and learnis on the fly.
Building blocks of the bayesian machine learning are as follows - 
- Predictive Distribution $p(y_i|x_i)$ - distribution we want to obtain
- Likelihood $p(y_i|x_i, \theta_i)$ - given the model parameters, how realistic is it to get the pair $(x_i, y_i)$
- Prior Distribution $p(\theta_i)$

Choosing a *good* prior is important. If not chosen properly we not get a tractable equation to update our model prameters and we'll need to depend upon techniques like **MCMC** and **variational inference**
- Posterior Distribution $p(\theta_{i+1}|\theta{i}, x_i, y_i) \propto p(y_i|x_i,\theta_i)p(\theta_i)$

Now the predictive distribution can be computed as  - 
$$
\int p(y_i|\boldsymbol{w}, x_i)p(\boldsymbol{w})d\boldsymbol{w}
$$
We have marginalized over the model parameters. This equation is intractible if we have not chosen the conjugate prior to the likelihood.

We’re computing a weighted average of the potential $y_i$ values for each possible model parameter $\boldsymbol{w}$

### Online Belief Updating 

Whenever a new pair of $(x_i, y_i)$ arrives we update the distribution of parameters - 
$$
p(\theta_{i+1}|\theta_{i}, x_i, y_i) \propto p(x_i, y_i|\theta_i)p(\theta_i)
$$

Before any data comes in and we are asked to predict the $y_0$ - 
$$
p(y_0|x_0) \propto p(y_0|x_0, \theta_0)p(\theta_0)
$$

Ones we see the first output $y_0$ and hence the pair $(x_0, y_0)$, we can update our model parameters - 
$$
p(\theta_1|\theta_0, x_0, y_0) \propto p(x_0, y_0|\theta_0)p(\theta_0)
$$

Now again the predictive distribution for the output $y_1$ is given by -
$$
p(y_1|x_1) \propto p(y_1|x_1, \theta_1)p(\theta_1|\theta_0, x_0, y_0)
$$

i.e. the prior of the weights of the current iteration is the posterior of the weights of the previous iteration.

Once we see the second output $y_1$, we can now update our prior again - 
$$
\begin{equation}
\begin{split}
p(\theta_2|\theta_1, x_1, y_1) & \propto p(y_1|x_1, \theta_1)p(\theta_1) \\
& \propto p(y_1|x_1, \theta_1)p(y_0|x_0, \theta_0)p(\theta_0)
\end{split}
\end{equation}
$$ 

similarly - 
$$
\begin{equation}
\begin{split}
p(\theta_3|\theta_2, x_1, y_1) &  \propto p(y_2|x_2, \theta_2)p(\theta_2) \\
& \propto p(y_2|x_2, \theta_2)p(y_1|x_1, \theta_1)p(y_0|x_0, \theta_0)p(\theta_0)
\end{split}
\end{equation}
$$

And so on and so forth. The posterior distribution at step $i+1$ becomes the prior at step $i$. **We only need to store the current distribution of weights to make everything work.**




### Linear Regression (Bayesian)

$$
y_i = w_i^Tx_i + \epsilon_i
$$

$y_i \in \boldsymbol{R}$,
$x_i \in \boldsymbol{R}^d$,
$w_i \in \boldsymbol{R}^d$

$\epsilon_i$ follows Gaussian Distribution. Hence the likelihood function is a gaussian distribution - 
$$
y_i|(x_i, w_i) \sim \mathcal{N}(w_i^Tx_i, \beta_i^{-1})
$$
$\beta_i$ - precision >0 

Prior distribution for the above likelihood function is the **multivariate gaussian distribtuion**
$$
w_0 \sim \mathcal{N}(m_0, S_0)
$$
$m_0$ is the mean of the distribution $\in \boldsymbol{R}^d$ <br>
$S_0$ is the covariance matrix $\in \boldsymbol{R}^{d*d}$

$$
\begin{align*}

m_0 & = (0, 0, ..., 0) \\
S_0 & = diag(\alpha^{-1}, \alpha^{-1}, ..., \alpha^{-1})
\end{align*}
$$

Now the posterior distribution of the weights is given by - 
$$
\begin{align*}
p(w_{i+1}|w_i, x_i, y_i) & = \mathcal{N}(m_{i+1}, S_{i+1}) \\
S_{i+1} & = (S_{i+1} + \beta x_ix_t^T)^{-1} \\
m_{i+1} & = S_{i+1}(S_{i}^{-1}m_i + \beta x_i^Ty_i)
\end{align*}
$$

And the predictive distribution is given as follows - 

$$
\begin{align*}
p(y_i) & = \mathcal{\mu_i, \sigma_i} \\
\mu_i & = w_i^Tx_i \\
\sigma_i & = \frac{1}{\beta} + x_i^TS_ix_i
\end{align*}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

class BayesLinReg:

    def __init__(self, n_features, alpha, beta):
        self.alpha = alpha
        self.beta = beta
        self.n_features = n_features
        self.mean = np.zeros(n_features)
        self.cov = (1 / alpha) * np.eye(n_features)

    def learn(self, x, y):
        # Update the covariance matrix
        cov_inv = self.cov_inv(x) + self.beta * np.outer(x, x)
        # Update the mean vector
        cov = np.linalg.inv(cov_inv)
        mean = cov @ (self.cov_inv(x) @ self.mean + self.beta * y * x)

        self.cov_inv = cov_inv
        self.mean = mean

        return self

    def predict(self, x):

        # Obtain the predictive mean (Bishop eq. 3.58)
        y_hat = x @ self.mean

        # obtain the predictive variance (Bishop eq. 3.59)
        w_cov = np.linalg.inv(self.cov_inv)
        y_var = 1 / self.beta + x @ w_cov @ x.T

        return stats.norm(y_hat, np.sqrt(y_var))

    @property
    def weights_dist(self):
        cov = np.linalg.inv(self.cov_inv)
        return stats.multivariate_normal(self.mean, cov)